In [ ]:
from itertools import zip_longest
import datetime
import os
import sys

import pandas as pd

import muscope

In [ ]:
print(sys.modules['muscope'].__file__)
muscope_dp = os.path.dirname(sys.modules['muscope'].__file__)
downloads_dp = os.path.join(muscope_dp, 'downloads')
caron_hl2a_vertprof_seq_attr_xls_fp = os.path.join(downloads_dp, 'Church_HOT201-222_Tricho16S_seq_assoc_v2.xls')

print(caron_hl2a_vertprof_seq_attr_xls_fp)
os.path.exists(caron_hl2a_vertprof_seq_attr_xls_fp)


In [ ]:
core_attr_plus_data_df = pd.read_excel(
    caron_hl2a_vertprof_seq_attr_xls_fp,
    sheet_name='core attributes + data',
    skiprows=(0,2)
)

In [ ]:
core_attr_plus_data_df.head()

In [ ]:
core_attr_plus_data_df.collection_date[0]

In [ ]:
core_attr_plus_data_df.collection_time[0]

In [ ]:
core_attr_plus_data_df.collection_date[0].time().isoformat(timespec='seconds')

In [ ]:
core_attr_plus_data_df.head()

In [ ]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [ ]:
for (r1, row1), (r2, row2) in grouper(core_attr_plus_data_df.iterrows(), n=2):
    # replace 'net tow' with '0' in the cast_num column
    if core_attr_plus_data_df.loc[r1, 'cast_num'] == 'net tow':
        core_attr_plus_data_df.loc[r1, 'cast_num'] = 0

    # add 'HOT' to cruise_name
    core_attr_plus_data_df.loc[r1, 'cruise_name'] = \
        'HOT' + str(int(core_attr_plus_data_df.loc[r1, 'cruise_name']))
        
    if r1 == 0:
        # copy time from first date column to first time column
        core_attr_plus_data_df.loc[0, 'collection_time'] = core_attr_plus_data_df.collection_date[0].time()
    else:
        # copy attributes from previous sample ONLY IF THE ATTRIBUTE IS EMPTY
        column_names = list(core_attr_plus_data_df.columns)
        # the depth column is missing some values but for now leave them blank
        column_names.remove('depth_sample')
        for attr_name in column_names:
            print('row {} attr "{}" is "{}"'.format(r1, attr_name, core_attr_plus_data_df.loc[r1, attr_name]))
            if str(core_attr_plus_data_df.loc[r1, attr_name]) in ('nan', 'NaT'):
                print('  copy "{}" from previous sample'.format(core_attr_plus_data_df.loc[r1-2, attr_name]))
                core_attr_plus_data_df.loc[r1, attr_name] = core_attr_plus_data_df.loc[r1-2, attr_name]
            else:
                pass

core_attr_plus_data_df

In [ ]:
core_attr_plus_data_df